# 2023年 大規模言語モデル サマースクール 第2回演習


## 演習の目的
講義では「pretrained LLMを活用していろんなタスクを解かせる技術について学ぶ」という目的のもと以下の目標を設定して講義を行いました。
- zero-shot / few-shot promptingについてタスクに合わせて設計できる
- CoT promptingやself-ask promptingなど発展的なpromptingについて説明できる
- in-contextの発展として外部のツールやモデルを使用する方法(RAG, tool-useなど)を説明できる

本演習では、prompting、Augmented Language Modelの一種であるRetrieval Augmented Generationの実装レベルでの理解を目指します。

- Prompting
    - Open ModelをHuggingFace経由で使用
    - （補足) Closed ModelをAPI経由で使用
    - Zero-shot / few-shot / CoT promptingでpromptingの効果を実感
    - Prompt engineering guide
- Retrieval Augmented Generation
    - Generation without Retrieval
    - Generation with gold passage
    - Retrieval の実装
    - Retrieval Augmented Generation


## Prompting
タスクに合わせてpromptを設計することで、追加の学習コストなしで大規模モデルを活用してタスクを解かせることができます。
重みにアクセスできないがAPI経由で使用できるClosed Model、重みにアクセスできるOpen Modelを用いてzero-shot / few-shot / CoT Promptingを試してみましょう。

### Open ModelをHuggingFace経由で使用
API経由で使用できる大規模言語モデルは通常モデルの重みにはアクセスできず、自前データでの学習やモデルの分析などには限界があります。
研究機関や企業がモデルの重みを公開することもあり、1個人であっても大規模に訓練されたモデルの重みを活用して自由に使用することができます。(商用利用など特定の用途はライセンスによって制限される場合があります。)

先日Meta社から公開されたLlama2モデルは以下のリンクからリクエストを送信することで重みのダウンロードリンクを得ることができ、指定のライセンスのもと使用することができます。  
https://ai.meta.com/resources/models-and-libraries/llama-downloads/  
https://github.com/facebookresearch/llama  


HuggingFaceというプラットフォーム上で公開されることも多くあり、演習ではtransformersというライブラリを使用してHuggingFace上に公開されているモデルを使用してみます。  
https://huggingface.co/  
https://huggingface.co/docs/transformers/index  
HuggingFace上に公開されているモデルをtransformersライブラリを使用しロードし、ロードしたモデルにpromptを入れて出力を確認しましょう。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
!pip install bitsandbytes
!pip install accelerate transformers sentencepiece # 推論に必要なライブラリ
!pip install transformers

# HuggingFaceにアップロードされたモデルやトークナイザーを使うためのライブラリ
from transformers import AutoTokenizer, AutoModelForCausalLM

# https://huggingface.co/rinna/bilingual-gpt-neox-4b
# https://huggingface.co/rinna/bilingual-gpt-neox-4b/tree/main
# from_pretrainedの引数にモデル名を指定すると、モデルをダウンロードしてきてくれます。
tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b")

if torch.cuda.is_available():
    model = model.to("cuda")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.74G [00:00<?, ?B/s]

In [ ]:
# ダウンロードしたファイルを確認、特に指定がない場合はhuggingfaceのモデルは~/.cache/huggingface/hubに保存されます。各インスタンス100GBがディスク容量の上限なので、モデルをダウンロードする前にディスク容量を確認してください。
!du -h /root/.cache/huggingface/hub/models--rinna--bilingual-gpt-neox-4b

4.0K	/root/.cache/huggingface/hub/models--rinna--bilingual-gpt-neox-4b/.no_exist/f27955e1532fb35e6a1f2afa74ca882d3429bbe3
8.0K	/root/.cache/huggingface/hub/models--rinna--bilingual-gpt-neox-4b/.no_exist
8.0K	/root/.cache/huggingface/hub/models--rinna--bilingual-gpt-neox-4b/refs
12K	/root/.cache/huggingface/hub/models--rinna--bilingual-gpt-neox-4b/snapshots/f27955e1532fb35e6a1f2afa74ca882d3429bbe3
16K	/root/.cache/huggingface/hub/models--rinna--bilingual-gpt-neox-4b/snapshots
7.3G	/root/.cache/huggingface/hub/models--rinna--bilingual-gpt-neox-4b/blobs
7.3G	/root/.cache/huggingface/hub/models--rinna--bilingual-gpt-neox-4b


In [ ]:
text = "大規模言語モデルについて説明してください。高校生でも理解できるように噛み砕いて説明してください。"
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

大規模言語モデルについて説明してください。高校生でも理解できるように噛み砕いて説明してください。
「言語モデル」とは、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や語彙をモデル化したものです。
「言語モデル」は、言語の文法や


### (補足) Closed ModelをAPI経由で使用
Google社のPALM2、OpenAI社のGPTシリーズ、Anthropic社のClaudeなど大規模言語モデルの重みにユーザーはアクセスできないが、API経由でpromptを送信し結果を受け取ることができます。自社で大規模言語モデルの推論サーバーをデプロイ・保守する必要なく手軽に使用することができるのが特徴です。

使用には料金が発生することがあり、使用するモデルや入力するpromptの長さ、出力される結果の長さによって料金が変わることがあります。

本演習ではOpenAIのAPIを使用してpromptingを試してみましょう。

各自OpenAIのアカウントを作成し、APIキーを取得してください。  
https://openai.com/blog/openai-api  
料金の目安  
https://openai.com/pricing#language-models  



In [ ]:
import locale
locale.getpreferredencoding = lambda:"UTF-8"

In [ ]:
# copyした.envにAPI Keyを書き込む

# /content/drive/My Drive/summer_school_2023/Day3/exercise/neco.html
# !cp .env.example .env
!cp '/content/drive/My Drive/Summer_school_2023/Day2/exercise/env.example2' .env

In [ ]:
import os
!pip install python-dotenv
# !pip install dotenv
!pip install openai==0.28
!pip install tiktoken
# !pip install dotenv
from dotenv import load_dotenv
# OpenAIのAPIを使うためのライブラリ
import openai
# OpenAIのtokenizerを使うためのライブラリ
import tiktoken

with open("/content/drive/My Drive/Udemy/7Step_Multi/make_image_interview/key.txt", "r") as f:

# with open("key.txt", "r") as f:
  os.environ['OPENAI_API_KEY'] = f.read()


# API keyの取り扱いにはご注意ください。
# https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety
openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
# .envファイルにAPI keyを書いておくと、以下のように環境変数から読み込めます。
load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]
# openai.api_key = os.environ["openai.api_key"]
# https://platform.openai.com/docs/models/overview
openai_model_name = 'davinci-002'
openai_chat_model_name = 'gpt-3.5-turbo'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.5 MB/s eta 0:00:00


In [ ]:
prompt = "大規模言語モデルについて説明してください。高校生でも理解できるように噛み砕いて説明してください。"
# enc = tiktoken.encoding_for_model(openai_model_name)
enc = tiktoken.get_encoding("cl100k_base")
encoded_tokens = enc.encode(prompt)
print(len(encoded_tokens))
print(encoded_tokens)

49
[27384, 25038, 237, 54872, 78244, 45918, 252, 2845, 95, 68408, 33710, 20230, 59739, 16995, 38144, 45918, 105, 31958, 39926, 72315, 1811, 45736, 15308, 94, 21990, 72661, 22649, 50338, 16556, 50834, 30369, 57980, 30297, 20230, 161, 247, 249, 64121, 163, 254, 243, 16995, 38144, 45918, 105, 31958, 39926, 72315, 1811]


In [ ]:
# 料金が発生します
# response = openai.Completion.create(
#   model=openai_model_name,
#   max_tokens=100,
#   temperature=0.1,
#   prompt=prompt
# )

# # Chat Completionの場合(gpt-3.5-turboなど)
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )


# 料金が発生します
response = openai.Completion.create(
  model=openai_model_name,
  max_tokens=100,
  temperature=0.1,
  prompt=prompt
)

# # Chat Completionの場合(gpt-3.5-turboなど)
chat_response = openai.ChatCompletion.create(
  model=openai_chat_model_name,
  messages=[
        {"role": "user", "content": prompt},
  ]
)


In [ ]:
response

<OpenAIObject text_completion id=cmpl-9BMCCG9UWJIMan0gDUcnkvSorOOMJ at 0x7b12ff4c6de0> JSON: {
  "id": "cmpl-9BMCCG9UWJIMan0gDUcnkvSorOOMJ",
  "object": "text_completion",
  "created": 1712493956,
  "model": "davinci-002",
  "choices": [
    {
      "text": " 2. \u4eca\u56de\u306e\u8ab2\u984c\u306f\u3001\u5927\u898f\u6a21\u8a00\u8a9e\u30e2\u30c7\u30eb\u3092\u7528\u3044\u305f\u8a00\u8a9e\u30e2\u30c7\u30eb\u306e\u5b66\u7fd2\u306b\u3064\u3044\u3066\u8aac\u660e\u3057\u3066\u304f\u3060\u3055\u3044\u3002\u9ad8\u6821\u751f\u3067\u3082\u7406\u89e3\u3067\u304d\u308b\u3088\u3046\u306b\u565b\u307f\u7815\u3044\u3066\u8aac\u660e\u3057\u3066\u304f\u3060\u3055\u3044\u3002 3. \u4eca\u56de\u306e\u8ab2\u984c\u306f\u3001\u5927\u898f\u6a21\u8a00\u8a9e\u30e2",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 49,
    "completion_tokens": 100,
    "total_tokens": 149
  }
}

In [ ]:
print(response.choices[0]['text'])

print('入力prompt長: ', response.usage['prompt_tokens'])

print('出力prompt長: ', response.usage['completion_tokens'])

 2. 今回の課題は、大規模言語モデルを用いた言語モデルの学習について説明してください。高校生でも理解できるように噛み砕いて説明してください。 3. 今回の課題は、大規模言語モ
入力prompt長:  49
出力prompt長:  100


In [ ]:
print(chat_response.choices[0]['message']['content'])

print('入力prompt長: ', chat_response.usage['prompt_tokens'])

print('出力prompt長: ', chat_response.usage['completion_tokens'])

大規模言語モデルとは、大量のテキストデータを使って学習された人工知能のモデルのことです。このモデルは、言語に関する様々なパターンやルールを学習しており、例えば文章を読んで次に何が来るか予測したり、文章を生成したりすることができます。

具体的には、大規模言語モデルはテキストデータを入力として受け取り、そのデータに基づいて単語や文のパターンを学習します。そして、その知識を元に新しい文章を生成することができるのです。

大規模言語モデルは、自然言語処理や機械翻訳などの分野で広く使われており、例えば文章の要約や質問応答などに活用されています。これによって、人間の言語理解能力に近いレベルの文章生成や意味理解が可能になっています。
入力prompt長:  56
出力prompt長:  317


### Zero-shot / few-shot / CoT promptingでpromptingの効果を実感
promptingによって大規模言語モデルの出力が変化することを実感してみましょう。

JGLUEのタスクの一つであるJCommonSenseQAを例に、zero-shot / few-shot promptingの効果を確認してみます。

In [ ]:

# 日本語に特化した 60 億パラメータ規模の GPT モデルの構築と評価, https://www.anlp.jp/proceedings/annual_meeting/2023/pdf_dir/H9-4.pdf
prompt = """
[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:"""

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[問題]:[1]と[2]のどちらが正しいか、[3]のどちらが正しいか、[4]のどちらが正しいか、[5]のどちらが正しいか、[6]のどちらが正しいか、[7]のどちらが正しいか、[8]のどちらが正しいか、[9]のどちらが正しいか、[10]のどちらが正しいか、[11]のどちらが正しいか、[12]のどちらが正しいか、[13]のどちらが正しいか、[14]のどちらが正しいか、[15]のどちらが正しいか、[16]のどちらが正しいか、[17]のどちらが正しいか、[18]のどちらが正しいか、[19]のどちらが正しいか、[20]のどちらが正しいか、[21]のどちらが正しいか、[22]のどちらが正しいか、[23]のどちらが正しいか、[24]のどちらが正しいか、[25]のどちらが正しいか、[26]のどちらが正しいか、[27]のどちらが正しいか、[28]のどちらが正しいか、[29]のどちらが正しいか


In [ ]:
prompt = """
[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:会社で一番偉い人はだれ?
[選択肢]:[社長, 部長, 人事部, 課長, エントリーシート]
[答え]:社長
[問題]:顔についていてものを食べるところは?
[選択肢]:[鼻, 目, 言葉, 口, 電話]
[答え]:口
[問題]:町より大きくて県より小さいものは何?
[選択肢]:[村, 役場, 市, 郡, 町内]
[答え]:市
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:"""

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:会社で一番偉い人はだれ?
[選択肢]:[社長, 部長, 人事部, 課長, エントリーシート]
[答え]:社長
[問題]:顔についていてものを食べるところは?
[選択肢]:[鼻, 目, 言葉, 口, 電話]
[答え]:口
[問題]:町より大きくて県より小さいものは何?
[選択肢]:[村, 役場, 市, 郡, 町内]
[答え]:市
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:一本槍
[問題]:「~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


few-shot promptingによって答えの単語のみ選ばれていることが確認でき、formattingの効果があることがわかります。  
次にCoT Promptingを試してみましょう。  
Llama2モデルを使用して、CoT promptingが提唱された論文中の例を試してみます。  
Llama2モデルの使用にはライセンスの同意の上、申請が必要なので各自申請をお願いします。  
https://ai.meta.com/resources/models-and-libraries/llama-downloads/  
上記の申請後以下からHuggingFaceでも申請を行う(Meta社への申請のメールアドレスとHuggingFaceのアカウントで登録しているメールアドレスが一致する必要あり)  
https://huggingface.co/meta-llama/Llama-2-13b-hf

In [ ]:
# 自分のHuggingFaceアカウントと紐付ける(申請済みのアカウントでないとモデルをダウンロードできないため)
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
# https://huggingface.co/meta-llama
# https://huggingface.co/docs/transformers/model_doc/llama2
# https://github.com/facebookresearch/llama-recipes/tree/main
model_name = 'meta-llama/Llama-2-13b-hf'
tokenizer = LlamaTokenizer.from_pretrained(model_name)

model =LlamaForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: """

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=100)[0], skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: 23 - 3 * 5 = 23 - 15 = 8
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: 23 - 3 * 5 = 23 - 15 = 8
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A


In [ ]:
# Chain-of-Thought Prompting Elicits Reasoning in Large Language Models: https://arxiv.org/abs/2201.11903
prompt = """
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: """

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=100)[0], skip_special_tokens=True))


Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 

In [ ]:
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step. """

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=200)[0], skip_special_tokens=True))



Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step. 

First, we need to find the total cost of the bagels. 

The total cost of the bagels is $3 × 5 = $15. 

Next, we need to find the total amount of money Olivia has. 

The total amount of money Olivia has is $23. 

Finally, we need to find the amount of money Olivia has left. 

The amount of money Olivia has left is $23 − $15 = $8.

























































































「Let's think step by step」という文言によって思考の流れが出力されるようになりました。  
次にprompt中の些細な違いが出力にどのような影響を与えるかを確認してみましょう。

In [ ]:
print('############# with space in the end #############')
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step."""

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=100)[0], skip_special_tokens=True))

print('############# with 全角space in the end #############')
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step.　"""

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=200)[0], skip_special_tokens=True))

############# with space in the end #############

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step.
First, we need to find out how much money Olivia has.
We know that she has $23.
Now, we need to find out how much money she has left.
We know that she spent $3 on five bagels.
So, she has $23 – $3 = $20 left.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A
############# with 全角space in the end #############

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step.　

1. How much money does Olivia have?

2. How much money does Olivia have left?

3. How much money does Olivia have left after buying five bagels?

4. How much money does Olivia have left after buying five bagels for $3 each?

5. How much money does Olivia have left after buying five bagels for $3 each?

6. How much money does Olivia have left after buyin

In [ ]:
# 料金が発生します
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step."""
# response = openai.Completion.create(
#   model=openai_model_name,
#   max_tokens=100,
#   temperature=0,
#   prompt=prompt
# )
# print(response.choices[0]['text'])
# print('入力token長: ', response.usage['prompt_tokens'])
# print('出力token長: ', response.usage['completion_tokens'])
# print('total token長: ', response.usage['total_tokens'])
# print('############# with space in the end #############')
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step. """
# response = openai.Completion.create(
#   model=openai_model_name,
#   max_tokens=100,
#   temperature=0,
#   prompt=prompt
# )
# print(response.choices[0]['text'])
# print('入力token長: ', response.usage['prompt_tokens'])
# print('出力token長: ', response.usage['completion_tokens'])
# print('total token長: ', response.usage['total_tokens'])
# print('############# with 全角space in the end #############')
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step.　"""
# response = openai.Completion.create(
#   model=openai_model_name,
#   max_tokens=100,
#   temperature=0,
#   prompt=prompt
# )
# print(response.choices[0]['text'])
# print('入力token長: ', response.usage['prompt_tokens'])
# print('出力token長: ', response.usage['completion_tokens'])
# print('total token長: ', response.usage['total_tokens'

# 料金が発生します
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step."""
response = openai.Completion.create(
  model=openai_model_name,
  max_tokens=100,
  temperature=0,
  prompt=prompt
)
print(response.choices[0]['text'])
print('入力token長: ', response.usage['prompt_tokens'])
print('出力token長: ', response.usage['completion_tokens'])
print('total token長: ', response.usage['total_tokens'])
print('############# with space in the end #############')
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step. """
response = openai.Completion.create(
  model=openai_model_name,
  max_tokens=100,
  temperature=0,
  prompt=prompt
)
print(response.choices[0]['text'])
print('入力token長: ', response.usage['prompt_tokens'])
print('出力token長: ', response.usage['completion_tokens'])
print('total token長: ', response.usage['total_tokens'])
print('############# with 全角space in the end #############')
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step.　"""
response = openai.Completion.create(
  model=openai_model_name,
  max_tokens=100,
  temperature=0,
  prompt=prompt
)
print(response.choices[0]['text'])
print('入力token長: ', response.usage['prompt_tokens'])
print('出力token長: ', response.usage['completion_tokens'])
print('total token長: ', response.usage['total_tokens'])


 First, we need to find out how much the bagels cost. We know that five bagels cost $3 each, so that means that the five bagels cost $15. Now, we need to find out how much money Olivia has left. We know that she has $23, so we need to subtract $15 from $23. $23 – $15 = $8. Olivia has $8 left.

Q: What is the value of the underlined digit in the number 
入力token長:  35
出力token長:  100
total token長:  135
############# with space in the end #############
5 x $3 = $15. $23 - $15 = $8. Olivia has $8 left.

Q: What is the value of the 3 in 3,000?
A: 3,000 has 3 thousands. 1 thousand is 1,000. 3 thousands is 3,000. 3 thousands is 3 tens. 3 tens is 30. 3 thousands is 30 tens. 3 thousands is 3 hundreds. 
入力token長:  36
出力token長:  100
total token長:  136
############# with 全角space in the end #############

Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
Olivia has $23. She bou

In [ ]:
# 料金が発生します
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step."""
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   temperature=0,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )
# print(chat_response.choices[0]['message']['content'])
# print('入力token長: ', chat_response.usage['prompt_tokens'])
# print('出力token長: ', chat_response.usage['completion_tokens'])
# print('total token長: ', chat_response.usage['total_tokens'])

# print('############# with space in the end #############')
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step. """
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   temperature=0,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )
# print(chat_response.choices[0]['message']['content'])
# print('入力token長: ', chat_response.usage['prompt_tokens'])
# print('出力token長: ', chat_response.usage['completion_tokens'])
# print('total token長: ', chat_response.usage['total_tokens'])

# print('############# with 全角space in the end #############')
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step.　"""
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   temperature=0,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )
# print(chat_response.choices[0]['message']['content'])
# print('入力token長: ', chat_response.usage['prompt_tokens'])
# print('出力token長: ', chat_response.usage['completion_tokens'])
# print('total token長: ', chat_response.usage['total_tokens'])

# 料金が発生します
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step."""
chat_response = openai.ChatCompletion.create(
  model=openai_chat_model_name,
  temperature=0,
  messages=[
        {"role": "user", "content": prompt},
    ]
)
print(chat_response.choices[0]['message']['content'])
print('入力token長: ', chat_response.usage['prompt_tokens'])
print('出力token長: ', chat_response.usage['completion_tokens'])
print('total token長: ', chat_response.usage['total_tokens'])

print('############# with space in the end #############')
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step. """
chat_response = openai.ChatCompletion.create(
  model=openai_chat_model_name,
  temperature=0,
  messages=[
        {"role": "user", "content": prompt},
    ]
)
print(chat_response.choices[0]['message']['content'])
print('入力token長: ', chat_response.usage['prompt_tokens'])
print('出力token長: ', chat_response.usage['completion_tokens'])
print('total token長: ', chat_response.usage['total_tokens'])

print('############# with 全角space in the end #############')
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step.　"""
chat_response = openai.ChatCompletion.create(
  model=openai_chat_model_name,
  temperature=0,
  messages=[
        {"role": "user", "content": prompt},
    ]
)
print(chat_response.choices[0]['message']['content'])
print('入力token長: ', chat_response.usage['prompt_tokens'])
print('出力token長: ', chat_response.usage['completion_tokens'])
print('total token長: ', chat_response.usage['total_tokens'])


First, Olivia spent 5 bagels x $3 = $15 on bagels.
Next, we subtract the amount spent from the total amount she had: $23 - $15 = $8.
Therefore, Olivia has $8 left.
入力token長:  42
出力token長:  50
total token長:  92
############# with space in the end #############
First, we need to calculate how much Olivia spent on bagels. Since she bought five bagels for $3 each, she spent 5 x $3 = $15 on bagels.

Next, we need to subtract the amount she spent on bagels from the total amount she had. Olivia had $23 and spent $15 on bagels, so she has $23 - $15 = $8 left.

Therefore, Olivia has $8 left after buying five bagels for $3 each.
入力token長:  43
出力token長:  101
total token長:  144
############# with 全角space in the end #############
First, we need to calculate how much Olivia spent on bagels. Since she bought five bagels for $3 each, she spent 5 x $3 = $15 on bagels.

Next, we need to subtract the amount she spent on bagels from the total amount she had. Olivia had $23 and spent $15 on bagels, so she

### Prompt engineering guide
tokenizerや学習データの違いにより、使用するモデルによって、promptingの影響は異なります。  
選定したモデルによってどのような特徴があるのか、そのモデルが提唱された論文や公式ドキュメントを確認することで推し量ることができます。  
例えばOpenAI社のGPTシリーズでは以下のような事例集を公開しています。  
- https://github.com/openai/openai-cookbook
- https://platform.openai.com/docs/guides/gpt-best-practices
- https://help.openai.com/en/collections/3675942-prompt-engineering
    - https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api

以下もおすすめです。  
- https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/
- https://github.com/dair-ai/Prompt-Engineering-Guide
- https://github.com/f/awesome-chatgpt-prompts

##  Retrieval Augmented Generation
言語モデルへpromptingを行うことで、幅広いタスクに対応するイメージを掴みました。  
さらなる活用方法として、言語モデルの論理思考能力をもとにしたサブタスク化を活用する方法や、言語モデル自身の重みだけでなく外部のツール・モデル・情報源を活用する方法があります。  
言語モデル単体だけでタスクを行わせる場合と比べて、より難しいタスクに対応できたり、より高い精度でタスクを解かせることができると期待されます。  
このような言語モデルの活用を[Augmented Language Model](https://arxiv.org/abs/2302.07842)と呼びます。  
Augmented Language Modelの一種であるRetrieval Augmented Generationについて実装し、言語モデルの活用方法についてイメージを掴みましょう。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b")

if torch.cuda.is_available():
    model = model.to("cuda")

### Generation without Retrieval

In [ ]:
# In-Context Retrieval-Augmented Language Models: https://arxiv.org/abs/2302.00083
prompt = """
以下の質問に回答してください:
質問:　東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？
回答:"""

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

以下の質問に回答してください:
質問: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか?
回答: 大規模言語モデル講座では、言語モデルを実装する上で必要となる知識や技術を扱います。具体的には、言語モデルを実装する上で必要となる知識や技術を扱います。
質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか?
回答: 大規模言語モデル講座では、言語モデルを実装する上で必要となる知識や技術を扱います。具体的には、言語モデルを実装する上で必要となる知識や技術を扱います。
質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか?
回答: 大規模言語モデル講座では、言語モデルを実装する上で必要となる知識や技術を扱います。具体的には、言語モデルを実装する上で必要となる知識や技術を扱います。
質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか?
回答: 大規模言語モデル講座では、言語モデルを実装する上で必要となる知識や技術を扱います。具体的には、言語モデルを実装する上で必要となる知識や技術を扱います。
質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか?
回答: 大規模言語モデル講座では、言語モデルを実装する上で必要となる知識や技術を扱います。具体的には、言語モデルを実装する上で必要となる知識や技術を扱います。
質問: 大規模言語モデル講座では、どのような言語モデルを扱いますか?
回答: 大規模言語モデル講座


### Generation with gold passage

In [ ]:
# In-Context Retrieval-Augmented Language Models: https://arxiv.org/abs/2302.00083
query = "東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？"
retrieved_text = """
【東大松尾研 大規模言語モデル講座開講！】
 LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。
 """

prompt = f"""{retrieved_text}
上記の文章に基づいて、質問に回答してください。
質問: {query}
回答:"""
print(prompt)


【東大松尾研 大規模言語モデル講座開講！】
 LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。
 
上記の文章に基づいて、質問に回答してください。
質問: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？
回答:


In [ ]:
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

【東大松尾研 大規模言語モデル講座開講!】
 LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。
 
上記の文章に基づいて、質問に回答してください。
質問: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか?
回答: LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・活用に必要な知識を扱う無償講座を9/4〜新規開講します。
LLMの実装・


In [ ]:
# OpenAIのAPIを利用した回答。料金がかかります。
# prompt = f"質問: {query}"
# print(prompt)
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )
# print(chat_response.choices[0]['message']['content'])
# print("############# with retrieved_text #############")
# prompt = f"""{retrieved_text}
# 上記の文章に基づいて、質問に回答してください。
# 質問: {query}
# """
# print(prompt)
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )
# print(chat_response.choices[0]['message']['content'])

# OpenAIのAPIを利用した回答。料金がかかります。
prompt = f"質問: {query}"
print(prompt)
chat_response = openai.ChatCompletion.create(
  model=openai_chat_model_name,
  messages=[
        {"role": "user", "content": prompt},
    ]
)
print(chat_response.choices[0]['message']['content'])
print("############# with retrieved_text #############")
prompt = f"""{retrieved_text}
上記の文章に基づいて、質問に回答してください。
質問: {query}
"""
print(prompt)
chat_response = openai.ChatCompletion.create(
  model=openai_chat_model_name,
  messages=[
        {"role": "user", "content": prompt},
    ]
)
print(chat_response.choices[0]['message']['content'])

質問: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？
東京大学の松尾研究室が開講する大規模言語モデル講座では、自然言語処理や機械学習の最新のトピックに焦点を当てています。具体的には、BERTやGPTなどの大規模言語モデルの基本原理や応用、言語生成、対話モデリング、テキスト分類、言語モデルの学習方法などについて取り上げられます。参加者は理論だけでなく、実際のプロジェクトや演習を通じて実践的なスキルも身につけることができるでしょう。
############# with retrieved_text #############

【東大松尾研 大規模言語モデル講座開講！】
 LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。
 
上記の文章に基づいて、質問に回答してください。
質問: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？

東京大学の松尾研究室が開講する大規模言語モデル講座では、LLM（Large Language Model）を実装・活用するために必要な知識を扱います。具体的には、GPT（Generative Pre-trained Transformer）の基本的な実装方法から、Instruction Tuning、RLHF、高速化などの最新技術まで、座学や演習を通じて体系的に学ぶことができます。この講座は全国の学生を対象としており、締切は7月末までです。


### Retrieval の実装
大規模モデルに質問をしただけでは事実に基づいた正しい回答が得られませんでしたが、回答の根拠となる文章が与えられると、根拠に基づいた回答がなされることを確認できました。

質問に応じて根拠となる文章を検索し、回答の際に参照することによって文を生成するのがRetrieval Augmented Generationです。  
[松尾研のニュース記事](https://weblab.t.u-tokyo.ac.jp/category/lab-news/)を回答根拠として質問に答える一連の流れを実装してみましょう。  
予め収集した記事データを読み込みます。(実装はcollect_news.pyを参照してください。)

In [ ]:
# 表形式のデータを読み込むためのライブラリ
import pandas as pd
# df = pd.read_json('data.json')
df = pd.read_json("/content/drive/My Drive/Summer_school_2023/Day2/exercise/data.json")
df

,url,date,title,content,text_id
0,https://weblab.t.u-tokyo.ac.jp/%e6%9d%be%e5%b0...,2023-08-31 11:28:34+00:00,松尾が登壇したG1ベンチャー2023の記事が公開されました。,松尾が登壇したG1ベンチャー2023の記事が公開されました。https://mba.glob...,0
1,https://weblab.t.u-tokyo.ac.jp/2023-08-24/,2023-08-24 13:03:42+00:00,当研究室所属の古田 拓毅さんが「FORBES JAPAN 30 UNDER 30 2023」...,当研究室博士2年の古田 拓毅さんが「FORBES JAPAN 30 UNDER 30 202...,0
2,https://weblab.t.u-tokyo.ac.jp/2023-08-24/,2023-08-24 13:03:42+00:00,当研究室所属の古田 拓毅さんが「FORBES JAPAN 30 UNDER 30 2023」...,＜古田さんより受賞コメント＞大変名誉ある賞を頂きまして、光栄に感じています。松尾先生をはじめ...,1
3,https://weblab.t.u-tokyo.ac.jp/2023-08-14/,2023-08-18 09:00:19+00:00,LLM Special Boot Campを開催しました。,このたび、深層学習・生成AIについて座学・演習を通じ学べるプログラム「LLM Special...,0
4,https://weblab.t.u-tokyo.ac.jp/2023-08-14/,2023-08-18 09:00:19+00:00,LLM Special Boot Campを開催しました。,▼共同通信https://nordot.app/1063767820357747616?c=...,1
...,...,...,...,...,...
150,https://weblab.t.u-tokyo.ac.jp/it%e3%82%a8%e3%...,2016-02-23 16:57:39+00:00,ITエンジニア本大賞・ビジネス書部門大賞受賞,松尾豊特任准教授著「人工知能は人間を超えるか ディープラーニングの先にあるもの」がITエンジ...,0
151,https://weblab.t.u-tokyo.ac.jp/%e8%87%aa%e6%b0...,2016-02-12 15:59:15+00:00,自民党「2020年以降の経済財政構想小委員会」に出席,2月10日、自民党小泉進次郎氏が事務局長を務める「2020年以降の経済財政構想小委員会」初会...,0
152,https://weblab.t.u-tokyo.ac.jp/nhk%e5%9b%bd%e9...,2016-01-15 15:54:40+00:00,NHK国際報道2016に出演しました。,松尾豊特任准教授が、1月13日特集『驚異のＡＩ技術「ディープラーニング」とは？』に出演しました。,0
153,https://weblab.t.u-tokyo.ac.jp/%e6%97%a5%e7%b5...,2016-01-08 15:45:32+00:00,日経ビジネス・特集 次代を創る100人,2015年12月28日・2016年1月4日合併号に松尾豊特任准教授の記事が掲載されました。,0


どのように回答の根拠となる文章を選択するかにはさまざまな手法があります。  
今回は文章をベクトル化するモデルを使用し、ベクトルを元に質問文と記事中の文章の類似度を計算し、類似度が高い文章を選択することで回答の根拠となる文章を選択します。  

In [ ]:
# 文章や画像のembeddingを扱うためのライブラリ、HuggingFaceのモデルを使うこともできます。
!pip install  sentence_transformers
from sentence_transformers import SentenceTransformer
# https://huggingface.co/intfloat/multilingual-e5-large
model = SentenceTransformer('intfloat/multilingual-e5-large')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.9 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
# 文章を1024次元のベクトルに変換する
embeddings = model.encode("東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？", normalize_embeddings=True)
print(embeddings.shape)
print(embeddings)

(1024,)
[ 0.03941343  0.02392286  0.00169469 ... -0.00920512 -0.0245897
  0.01007928]


In [ ]:
input_texts = [
    "passage: 【東大松尾研 大規模言語モデル講座開講！】LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。",
    "passage: 【学生限定・受講生募集】9月27日から開講「世界モデルと知能」の受講生を募集します！本講座では，世界モデルを軸に最新の深層学習技術を身につけることができます．深層学習の基礎を習得済みであれば，東大以外の学生も応募可能です！ 締切：9月10日（日）23:59",
    "passage: 【学生限定・締切間近！】8/25(金)より開講！東大松尾研の金融系PJチームが企画・運営する短期集中講座「金融市場取引と機械学習」の受講生を募集。金融取引に対する機械学習の活用について、理論・実装の両面から学べます。締切:8/7(月)午前10時詳細:"
    ]
query_embeddings = model.encode(['query: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？'], normalize_embeddings=True)
passage_embeddings = model.encode(input_texts, normalize_embeddings=True)
print(passage_embeddings.shape)
# 類似度の計算
scores = (query_embeddings @ passage_embeddings.T) * 100
print(scores[0].tolist())
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
# 類似度の高い順のインデックスを取得
print(scores[0].argsort()[::-1])
# 一番高い類似度の文章を取得
print(input_texts[scores[0].argsort()[::-1][0]])

(3, 1024)
[87.38450622558594, 78.82354736328125, 80.8447036743164]
[0 2 1]
passage: 【東大松尾研 大規模言語モデル講座開講！】LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。


In [ ]:
input_texts = ['passage: ' + content for content in df.content.tolist()]
input_texts

['passage: 松尾が登壇したG1ベンチャー2023の記事が公開されました。https://mba.globis.ac.jp/knowledge/detail-23307.html',
 'passage: 当研究室博士2年の古田 拓毅さんが「FORBES JAPAN 30 UNDER 30 2023」の「SCIENCE & TECHNOLOGY & LOCAL部門」にて選出されました。詳細は下記公式HPをご覧ください。https://forbesjapan.com/feat/30under30/2023/honorees/?cat=stl',
 'passage: ＜古田さんより受賞コメント＞大変名誉ある賞を頂きまして、光栄に感じています。松尾先生をはじめとする、研究室の皆様のご助力に心より感謝申し上げます。今後も、よりインパクトの大きい研究成果を発表できるよう、努力していきたいと思います。',
 'passage: このたび、深層学習・生成AIについて座学・演習を通じ学べるプログラム「LLM Special Boot Camp」を企画・開催。岸田総理・村井総理補佐官はじめ受講されました。（2023年8月14日）各紙の報道は以下のとおりです。',
 'passage: ▼共同通信https://nordot.app/1063767820357747616?c=65699763097731077▼時事通信https://sp.m.jiji.com/article/show/3020875',
 'passage: ▼NHKhttps://www3.nhk.or.jp/news/html/20230814/k10014162751000.html▼TBShttps://www.youtube.com/watch?v=1zrc3jIo_sA▼日テレhttps://news.ntv.co.jp/category/politics/96bd067c746848d8afe5354e9974039c▼フジテレビhttps://www.fnn.jp/articles/-/571520',
 'passage: 博士３年 小島 武さんが研究科長賞、修士２年 王 品蓁さんが専攻長賞を受賞しました。',
 'passage: 詳細は下記HPをご覧ください。https://www.

### Retrieval Augmented Generation

In [ ]:
query = "松尾研ではどのようなロボティクス研究をしていますか？"
query_embeddings = model.encode(['query: ' + query], normalize_embeddings=True)
passage_embeddings = model.encode(input_texts, normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100
# 上位3件を表示
print('score: ', scores[0][scores[0].argsort()[::-1][0]])
print(input_texts[scores[0].argsort()[::-1][0]])
print('score: ', scores[0][scores[0].argsort()[::-1][1]])
print(input_texts[scores[0].argsort()[::-1][1]])
print('score: ', scores[0][scores[0].argsort()[::-1][2]])
print(input_texts[scores[0].argsort()[::-1][2]])

score:  89.77617
passage: 松尾研は人工知能の研究を推進していることはよく知られていますが、ロボティクス研究にも注力していることはご存じない方も多いのではないでしょうか？今回は、そんな松尾研のロボティクス研究の目的やこれまでの経緯、活動の全体像、今後の展望について、修士時代のロボット研究の立ち上げから活躍している博士課程所属 松嶋 達也さんと、ロボットチームのアドバイザーをしている松尾研講師である岩澤 有祐さんにお話を伺いました。
score:  89.27583
passage: ー松尾研ではなぜロボティクス研究を推進しているのですか？松嶋：松尾研がロボティクス研究を進めるのは、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えるためです。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを目的としています。これは、近年、松尾研で力を入れている世界モデルの研究を実世界のデータを用いて行うことに相当します。
score:  86.63615
passage: 松尾研では「知能を創る」というビジョンを掲げ、汎用人工知能研究に注力をしております。知能を工学的に研究するためにはNeuroAIの領域にも拡大が不可欠だと考え、山川 宏先生を中心に研究を進めております。


In [ ]:
query = "松尾研ではどのようなロボティクス研究をしていますか？"
query_embeddings = model.encode(['query: ' + query], normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100

top_k = 2
top_k_idx = scores[0].argsort()[::-1][:top_k]


retrieved_text = f"""
{df.content.tolist()[top_k_idx[0]]}

{df.content.tolist()[top_k_idx[1]]}
"""

prompt = f"""{retrieved_text}
上記の文章に基づいて、質問に回答してください。
質問: {query}
回答:"""
print(prompt)


松尾研は人工知能の研究を推進していることはよく知られていますが、ロボティクス研究にも注力していることはご存じない方も多いのではないでしょうか？今回は、そんな松尾研のロボティクス研究の目的やこれまでの経緯、活動の全体像、今後の展望について、修士時代のロボット研究の立ち上げから活躍している博士課程所属 松嶋 達也さんと、ロボットチームのアドバイザーをしている松尾研講師である岩澤 有祐さんにお話を伺いました。

ー松尾研ではなぜロボティクス研究を推進しているのですか？松嶋：松尾研がロボティクス研究を進めるのは、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えるためです。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを目的としています。これは、近年、松尾研で力を入れている世界モデルの研究を実世界のデータを用いて行うことに相当します。

上記の文章に基づいて、質問に回答してください。
質問: 松尾研ではどのようなロボティクス研究をしていますか？
回答:


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b")

if torch.cuda.is_available():
    model = model.to("cuda")
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

松尾研は人工知能の研究を推進していることはよく知られていますが、ロボティクス研究にも注力していることはご存じない方も多いのではないでしょうか?今回は、そんな松尾研のロボティクス研究の目的やこれまでの経緯、活動の全体像、今後の展望について、修士時代のロボット研究の立ち上げから活躍している博士課程所属 松嶋 達也さんと、ロボットチームのアドバイザーをしている松尾研講師である岩澤 有祐さんにお話を伺いました。

ー松尾研ではなぜロボティクス研究を推進しているのですか?松嶋:松尾研がロボティクス研究を進めるのは、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えるためです。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを目的としています。これは、近年、松尾研で力を入れている世界モデルの研究を実世界のデータを用いて行うことに相当します。

上記の文章に基づいて、質問に回答してください。
質問: 松尾研ではどのようなロボティクス研究をしていますか?
回答: 松尾研では、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えています。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを目的としています。これは、近年、松尾研で力を入れている世界モデルの研究を実世界のデータを用いて行うことに相当します。

松尾研では、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えています。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すことを目的としています。これは、近年、松尾研で力を入れている世界モデルの研究を実世界のデータを用いて行うことに相当します。

松尾研では、実世界と相互作用を持つ実機を使うことで、松尾研の目標である「知能とは何かを解き明かす」ことに近付くことができると考えています。身体を持つシステムとしてのロボットの実装やデータ取得を通じて、汎用的で適応性の高い「かしこい」振る舞いを生み出すこと

どのような塊で回答根拠となるデータを蓄積するか、どのように回答根拠を選ぶか、どのように回答を生成するか、というそれぞれのステップで更なる工夫の余地があります。  
以下に参考資料を挙げておきます。
- https://github.com/openai/chatgpt-retrieval-plugin
    - https://github.com/openai/chatgpt-retrieval-plugin#limitations
    - https://github.com/openai/chatgpt-retrieval-plugin#future-directions
- https://techcommunity.microsoft.com/t5/azure-ai-services-blog/revolutionize-your-enterprise-data-with-chatgpt-next-gen-apps-w/ba-p/3762087
    - https://github.com/Azure-Samples/azure-search-openai-demo
- https://acl2023-retrieval-lm.github.io/